In [0]:
# ——————————————————————————————
#  COMMON SETUP
# ——————————————————————————————
# dbutils.widgets.text("pipeline_env", "ondra_test")
# env = dbutils.widgets.get("pipeline_env")
env = dbutils.widgets.get("pipeline.env")
print(f"Job environment: {env}")
catalog       = "principal_lab_db"
bronze_schema = f"{env}_bronze"
silver_schema = f"{env}_silver"
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {silver_schema}")

In [0]:
# ————————————————————————————————————————————————
# FINAL NON-STREAMING TABULKY PRO STREAMING
# ————————————————————————————————————————————————

def promote_to_batch_table(table_name: str):
    """
    Funkce vytváří nové tabulky ze silver tabulek.
    """
    source_path = f"{catalog}.{silver_schema}.{table_name}"
    target_path = f"{catalog}.{silver_schema}.{table_name}_mask"
    
    (
        spark.read.table(source_path)
             .write
             .mode('overwrite')
             .option('overwriteSchema', 'true')
             .saveAsTable(target_path)
    )

# Seznam tabulek pro maskování
tables_to_promote = [
    'dim_agents',
    'dim_customers',
    'dim_policies',
    'dim_products',
    'fact_claims',
    'fact_premium_transactions'
]

for table in tables_to_promote:
    promote_to_batch_table(table)